In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!which conda

!conda create -n pmp -c conda-forge pymeep=*=mpi_mpich_* -y

!conda run -n pmp pip install autograd
!conda run -n pmp pip install nlopt
!conda run -n pmp pip install "numpy<2" --no-deps --force-reinstall

!conda run -n pmp pip install ipykernel

!conda run -n pmp python -m ipykernel install --user --name pmp --display-name "Python (pmp)"

✨🍰✨ Everything looks OK!
/usr/local/bin/conda
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/pmp

  added / updated specs:
    - pymeep[build=mpi_mpich_*]


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  attr               conda-forge/linux-64::attr-2.5.1-h166bdaf_1 
  brotli             conda-forge/linux-64::brotli-1.1.0-hb9d3cd8_2 
  brotli-bin         conda-forge/linux-64::brotli-bin-1.1.0-hb9d3cd8_2 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h4bc722e_7 
  c-ares             conda-forge/linux-64::c-ares-1.34.4-hb9d3cd8_0 
  ca-certificates    conda-fo

In [4]:
%%bash
cat > script.py <<'EOF'
#!/usr/bin/env python
"""
Script for running Meep simulations with additional custom code.
Insert your custom Meep simulation code in the designated section below.
"""

import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
import nlopt
from matplotlib import pyplot as plt

# Print version information for verification
print("meep version:", mp.__version__)
print("numpy version:", np.__version__)

# ============================================================
# Insert your custom Meep code below
# ============================================================
# ---- Begin Meep code ----
#
# Add your custom Meep simulation code here.
#
# ---- End Meep code ----
# ============================================================
EOF

# Optional: display the contents of the generated script file
conda run -n pmp python -u script.py | tee output.txt

Using MPI version 4.1, 1 processes
meep version: 1.29.0
numpy version: 1.26.4

Elapsed run time = 0.2680 s



In [9]:
%%bash
cat > script.py <<'EOF'
#!/usr/bin/env python
"""
Script for running Meep simulations with additional custom code.
Insert your custom Meep simulation code in the designated section below.
"""

import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
import nlopt
from matplotlib import pyplot as plt

# Print version information for verification
print("meep version:", mp.__version__)
print("numpy version:", np.__version__)

# ============================================================
# Insert your custom Meep code below
# ============================================================

import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
from matplotlib import pyplot as plt

def J(alpha):
    return npa.abs(alpha) ** 2

mp.verbosity(0)
seed = 240
np.random.seed(seed)
Si = mp.Medium(index=3.4)
SiO2 = mp.Medium(index=1.44)

resolution = 20

Sx = 6
Sy = 5
cell_size = mp.Vector3(Sx, Sy)

pml_layers = [mp.PML(1.0)]

fcen = 1 / 1.55
width = 0.1
fwidth = width * fcen
source_center = [-1, 0, 0]
source_size = mp.Vector3(0, 2, 0)
kpoint = mp.Vector3(1, 0, 0)
src = mp.GaussianSource(frequency=fcen, fwidth=fwidth)
source = [
    mp.EigenModeSource(
        src,
        eig_band=1,
        direction=mp.NO_DIRECTION,
        eig_kpoint=kpoint,
        size=source_size,
        center=source_center,
    )
]

design_region_resolution = 10
Nx = design_region_resolution + 1
Ny = design_region_resolution + 1

design_variables = mp.MaterialGrid(mp.Vector3(Nx, Ny), SiO2, Si, grid_type="U_MEAN")
design_region = mpa.DesignRegion(
    design_variables, volume=mp.Volume(center=mp.Vector3(), size=mp.Vector3(1, 1, 0))
)


geometry = [
    mp.Block(
        center=mp.Vector3(x=-Sx / 4), material=Si, size=mp.Vector3(Sx / 2, 0.5, 0)
    ),  # horizontal waveguide
    mp.Block(
        center=mp.Vector3(y=Sy / 4), material=Si, size=mp.Vector3(0.5, Sy / 2, 0)
    ),  # vertical waveguide
    mp.Block(
        center=design_region.center, size=design_region.size, material=design_variables
    ),  # design region
    # mp.Block(center=design_region.center, size=design_region.size, material=design_variables,
    #        e1=mp.Vector3(x=-1).rotate(mp.Vector3(z=1), np.pi/2), e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), np.pi/2))
    #
    # The commented lines above impose symmetry by overlapping design region with the same design variable. However,
    # currently there is an issue of doing that; We give an alternative approach to impose symmetry in later tutorials.
    # See https://github.com/NanoComp/meep/issues/1984 and https://github.com/NanoComp/meep/issues/2093
]

sim = mp.Simulation(
    cell_size=cell_size,
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=source,
    eps_averaging=False,
    resolution=resolution,
)

TE0 = mpa.EigenmodeCoefficient(
    sim, mp.Volume(center=mp.Vector3(0, 1, 0), size=mp.Vector3(x=2)), mode=1
)
ob_list = [TE0]

opt = mpa.OptimizationProblem(
    simulation=sim,
    objective_functions=J,
    objective_arguments=ob_list,
    design_regions=[design_region],
    fcen=fcen,
    df=0,
    nf=1,
)

x0 = np.random.rand(Nx * Ny)
try :
    opt.update_design([x0])

except Exception as e:
    print(e)

f0, dJ_du = opt()


# ============================================================
# Plotting: visualize the dJ_du output as a 2D image
# ============================================================

# Ensure that dJ_du can be reshaped to (Nx, Ny)
try:
    reshaped = dJ_du.reshape(Nx, Ny)
except Exception as e:
    print("Error reshaping dJ_du:", e)
    reshaped = None

if reshaped is not None:
    plt.figure()
    plt.imshow(np.rot90(reshaped))
    plt.title("dJ_du Visualization")
    plt.colorbar()
    plt.savefig("dJ_du_plot.png")  # Save plot to file
    plt.close()
    print("Plot saved to 'dJ_du_plot.png'")
else:
    print("Could not generate plot due to reshaping error.")

# ============================================================
EOF

# Optional: display the contents of the generated script file
conda run -n pmp python -u script.py | tee output.txt

Using MPI version 4.1, 1 processes
meep version: 1.29.0
numpy version: 1.26.4
Starting forward run...
Starting adjoint run...
Calculating gradient...
Plot saved to 'dJ_du_plot.png'

Elapsed run time = 11.1462 s

